## 8. Summary

This notebook demonstrates a **DKW-guided fusion/fission controller** that:

### Key Features:
- **Statistical Guarantees**: Uses DKW inequality for confidence bounds on error rates
- **Adaptive Behavior**: Switches between fusion/fission modes based on observed performance
- **Hysteresis**: Prevents oscillation between modes
- **Parameter Tuning**: Adjustable thresholds and confidence levels

### Main Components:
1. **DKWController Class**: Core implementation with statistical guarantees
2. **Sample Data Generation**: Creates realistic test scenarios with varying difficulties
3. **Experiment Framework**: Compares proposed method against baseline
4. **Analysis & Visualization**: Interactive exploration of results
5. **Parameter Testing**: Ability to modify controller behavior

### Usage:
- Run all cells to see the complete analysis
- Modify parameters in Section 6 to explore different behaviors
- The notebook is completely self-contained - no external files needed
- All data and results are generated and analyzed within the notebook

### Next Steps:
- Try different parameter combinations in the interactive section
- Modify the sample data generation to test edge cases
- Implement additional baseline methods for comparison
- Add more sophisticated visualization and analysis

In [ ]:
# Optional: Save results to JSON file (uncomment to use)
# with open("method_out.json", "w") as f:
#     json.dump(results, f, indent=2)
# print("Results saved to method_out.json")

# Display sample results in the format that would be saved
print("Sample results (first 3 examples):")
print("\nBaseline:")
for i in range(3):
    print(f"  {results['baseline'][i]}")

print("\nProposed:")
for i in range(3):
    print(f"  {results['proposed'][i]}")
    
# Show the example output that was provided in the original question
print("\n" + "="*50)
print("Expected output format (from original method_out.json):")
example_output = {
    "baseline": [
        {"id": "example_000", "decision": "fission", "error": False},
        {"id": "example_001", "decision": "fission", "error": False},
        {"id": "example_002", "decision": "fission", "error": True}
    ],
    "proposed": [
        {"id": "example_000", "decision": "fission", "error": False},
        {"id": "example_001", "decision": "fusion", "error": False},
        {"id": "example_002", "decision": "fusion", "error": True}
    ]
}
print(json.dumps(example_output, indent=2))

## 7. Save Results (Optional)

If you want to save the results to JSON files (like the original script), run the cell below:

In [ ]:
# Interactive parameter exploration
def test_controller_params(epsilon_target=0.10, delta=0.05, min_samples=100, hysteresis=0.05):
    """Test controller with different parameters."""
    # Create a controller with custom parameters
    custom_controller = DKWController(
        epsilon_target=epsilon_target,
        delta=delta,
        min_samples=min_samples,
        hysteresis=hysteresis
    )
    
    # Run a quick experiment with subset of data
    test_data = sample_data[:150]  # Use first 150 examples for faster testing
    custom_results = {"baseline": [], "proposed": []}
    
    for example in test_data:
        error = np.random.random() < example["difficulty"]
        custom_controller.add_observation(float(error))
        decision = custom_controller.decide()
        
        custom_results["proposed"].append({
            "id": example["id"],
            "decision": decision,
            "error": error,
        })
    
    # Calculate metrics
    proposed_df = pd.DataFrame(custom_results["proposed"])
    metrics = calculate_metrics(proposed_df)
    
    print(f"Parameters: ε_target={epsilon_target}, δ={delta}, min_samples={min_samples}, hysteresis={hysteresis}")
    print(f"Results: Error rate={metrics['error_rate']:.3f}, Fusion rate={metrics['fusion_rate']:.3f}")
    print(f"Fusion decisions: {metrics['fusion_decisions']} out of {metrics['total_examples']}")
    
    return custom_results

# Test default parameters
print("=== DEFAULT PARAMETERS ===")
default_results = test_controller_params()

print("\n=== MODIFIED PARAMETERS ===")
# Try more aggressive parameters (lower target, less hysteresis)
aggressive_results = test_controller_params(epsilon_target=0.05, hysteresis=0.02)

# Try more conservative parameters (higher target, more hysteresis)
conservative_results = test_controller_params(epsilon_target=0.15, hysteresis=0.08)

## 6. Interactive Parameter Exploration

Try modifying the controller parameters below to see how they affect the behavior:

In [ ]:
# Visualization of controller behavior
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))

# 1. Decision patterns over time
decision_numeric = [1 if d == "fusion" else 0 for d in decisions_over_time]
ax1.plot(decision_numeric, alpha=0.7, linewidth=0.8)
ax1.set_title("Decision Pattern Over Time\n(1=Fusion, 0=Fission)")
ax1.set_xlabel("Example Number")
ax1.set_ylabel("Decision")
ax1.grid(True, alpha=0.3)

# 2. Cumulative fusion rate
fusion_rate_over_time = [fusion_count[i] / (i+1) for i in range(len(fusion_count))]
ax2.plot(fusion_rate_over_time, color='orange')
ax2.axhline(y=0.1, color='red', linestyle='--', alpha=0.7, label='Target ε=0.1')
ax2.set_title("Cumulative Fusion Rate")
ax2.set_xlabel("Example Number")
ax2.set_ylabel("Fusion Rate")
ax2.legend()
ax2.grid(True, alpha=0.3)

# 3. Error distribution by decision type
baseline_errors = baseline_df.groupby('decision')['error'].sum()
proposed_errors = proposed_df.groupby('decision')['error'].sum()

decisions = ['fission', 'fusion']
baseline_vals = [baseline_errors.get(d, 0) for d in decisions]
proposed_vals = [proposed_errors.get(d, 0) for d in decisions]

x = np.arange(len(decisions))
width = 0.35
ax3.bar(x - width/2, baseline_vals, width, label='Baseline', alpha=0.8)
ax3.bar(x + width/2, proposed_vals, width, label='Proposed', alpha=0.8)
ax3.set_title("Total Errors by Decision Type")
ax3.set_xlabel("Decision")
ax3.set_ylabel("Number of Errors")
ax3.set_xticks(x)
ax3.set_xticklabels(decisions)
ax3.legend()
ax3.grid(True, alpha=0.3)

# 4. Difficulty distribution
difficulties = [ex['difficulty'] for ex in sample_data]
ax4.hist(difficulties, bins=30, alpha=0.7, color='green')
ax4.set_title("Distribution of Example Difficulties")
ax4.set_xlabel("Difficulty Level")
ax4.set_ylabel("Frequency")
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Expected output format (from original data_out.json)
expected_format = [
    {
        "id": "example_000",
        "question": "What is 2+2?",
        "answer": "4",
        "difficulty": 0.15
    },
    {
        "id": "example_001",
        "question": "If x=5, what is 2x?",
        "answer": "10",
        "difficulty": 0.22
    },
    {
        "id": "example_002",
        "question": "Solve: 3y + 6 = 15",
        "answer": "y=3",
        "difficulty": 0.28
    }
]

print("Expected format structure:")
print(json.dumps(expected_format, indent=2))

print(f"\n📋 Note: Our processed data follows the same structure with ID, question, answer, and difficulty fields.")

In [ ]:
# Convert results to DataFrames for easier analysis
baseline_df = pd.DataFrame(results["baseline"])
proposed_df = pd.DataFrame(results["proposed"])

# Calculate metrics
def calculate_metrics(df):
    total_examples = len(df)
    errors = df["error"].sum()
    fission_decisions = (df["decision"] == "fission").sum()
    fusion_decisions = (df["decision"] == "fusion").sum()
    
    error_rate = errors / total_examples
    fission_rate = fission_decisions / total_examples
    fusion_rate = fusion_decisions / total_examples
    
    return {
        "total_examples": total_examples,
        "errors": errors,
        "error_rate": error_rate,
        "fission_decisions": fission_decisions,
        "fusion_decisions": fusion_decisions,
        "fission_rate": fission_rate,
        "fusion_rate": fusion_rate
    }

baseline_metrics = calculate_metrics(baseline_df)
proposed_metrics = calculate_metrics(proposed_df)

print("=== METRICS COMPARISON ===")
print(f"Baseline  - Error rate: {baseline_metrics['error_rate']:.3f}, Fission rate: {baseline_metrics['fission_rate']:.3f}")
print(f"Proposed  - Error rate: {proposed_metrics['error_rate']:.3f}, Fission rate: {proposed_metrics['fission_rate']:.3f}")
print(f"Difference - Error rate: {proposed_metrics['error_rate'] - baseline_metrics['error_rate']:+.3f}, Fission rate: {proposed_metrics['fission_rate'] - baseline_metrics['fission_rate']:+.3f}")

# Show how decisions evolved over time for proposed method
decisions_over_time = [r["decision"] for r in results["proposed"]]
fusion_count = [decisions_over_time[:i+1].count("fusion") for i in range(len(decisions_over_time))]
print(f"\nFusion decisions over time (final: {fusion_count[-1]} out of {len(decisions_over_time)})")

## Expected Output Format Reference

For reference, here's the expected output format that matches the original `data_out.json`:

## 5. Results Analysis

Let's analyze the results and visualize how the DKW controller performs compared to the baseline approach.

In [ ]:
# Demonstrate JSON serialization (in original script, this would save to file)
json_output = json.dumps(data, indent=2)
print("JSON format output:")
print(json_output)

# In the original script, this would be:
# with open("data_out.json", "w") as f:
#     json.dump(data, f, indent=2)
print(f"\n✓ Data collection complete! Processed {len(data)} examples.")

In [ ]:
def run_experiment(data):
    """Run DKW controller experiment with inline data."""
    controller = DKWController()
    results = {"baseline": [], "proposed": []}

    for example in data:
        # Simulate error occurrence based on difficulty
        error = np.random.random() < example["difficulty"]
        controller.add_observation(float(error))
        decision = controller.decide()

        results["proposed"].append({
            "id": example["id"],
            "decision": decision,
            "error": error,
        })
        results["baseline"].append({
            "id": example["id"],
            "decision": "fission",  # Always conservative
            "error": error,
        })

    return results

# Run the experiment
print("Running experiment...")
results = run_experiment(sample_data)

print(f"Processed {len(results['baseline'])} examples")
print(f"Baseline decisions: {len([r for r in results['baseline'] if r['decision'] == 'fission'])} fission, {len([r for r in results['baseline'] if r['decision'] == 'fusion'])} fusion")
print(f"Proposed decisions: {len([r for r in results['proposed'] if r['decision'] == 'fission'])} fission, {len([r for r in results['proposed'] if r['decision'] == 'fusion'])} fusion")

## Save Data (Optional)

In the original script, data would be saved to a JSON file. Here we'll demonstrate this functionality:

In [ ]:
# Display all processed data
print("Complete processed dataset:")
print("=" * 50)
for i, example in enumerate(data):
    print(f"\nExample {i+1}:")
    print(f"ID: {example['id']}")
    print(f"Question: {example['question']}")
    print(f"Answer: {example['answer']}")
    print(f"Difficulty: {example['difficulty']:.2f}")
    print("-" * 30)

## 4. Experiment Function

The experiment compares two approaches:
1. **Baseline**: Always uses "fission" mode (conservative approach)
2. **Proposed**: Uses DKW controller to adaptively switch between fusion/fission

For each example, we simulate whether an error occurs based on the difficulty level, then record the controller's decision.

## Display All Processed Data

Let's view the complete processed dataset:

In [ ]:
# Execute the data collection
data = collect_data()

print(f"Collected {len(data)} examples")
print("\nFirst example:")
print(json.dumps(data[0], indent=2))

In [ ]:
# Create sample data inline (replaces reading from JSON file)
# Each example has an ID and a difficulty level (0.0 to 1.0)
# Higher difficulty means higher probability of error
sample_data = [
    {"id": f"example_{i:03d}", "difficulty": np.random.beta(2, 8)}  # Most examples easy
    for i in range(200)
] + [
    {"id": f"example_{i+200:03d}", "difficulty": np.random.beta(5, 3)}  # Some harder examples
    for i in range(100)
] + [
    {"id": f"example_{i+300:03d}", "difficulty": np.random.beta(8, 2)}  # Few very hard examples
    for i in range(50)
]

print(f"Created {len(sample_data)} examples")
print(f"Difficulty range: {min(ex['difficulty'] for ex in sample_data):.3f} - {max(ex['difficulty'] for ex in sample_data):.3f}")
print(f"Mean difficulty: {np.mean([ex['difficulty'] for ex in sample_data]):.3f}")

# Show first few examples
print("\nFirst 5 examples:")
for i, example in enumerate(sample_data[:5]):
    print(f"  {example['id']}: difficulty = {example['difficulty']:.3f}")

## Execute Data Collection

Now let's run the data collection function and see the processed results.

## 3. Sample Data

Instead of reading from external JSON files, we'll create sample data inline. This data represents examples with varying difficulty levels that determine the probability of errors.

In [ ]:
def collect_data(dataset=None):
    """Collect benchmark data for DKW controller evaluation."""
    # Use sample data if no dataset provided (for self-contained version)
    if dataset is None:
        dataset = sample_dataset
    
    data = []
    for i, example in enumerate(dataset):
        data.append({
            "id": f"example_{i:03d}",
            "question": example["question"],
            "answer": example["answer"],
            "difficulty": len(example["question"]) / 100,  # Simple proxy based on question length
        })

    return data

print("Data processing function defined!")

## Data Processing Function

The `collect_data()` function processes the raw dataset and converts it to a standardized format for the DKW benchmark. It adds unique IDs and calculates a difficulty score based on question length.

In [ ]:
@dataclass
class DKWController:
    """DKW-guided fusion/fission controller."""
    epsilon_target: float = 0.10
    delta: float = 0.05
    min_samples: int = 100
    hysteresis: float = 0.05

    samples: list = field(default_factory=list)
    current_state: str = "fission"

    def dkw_epsilon(self, n: int) -> float:
        """Compute DKW epsilon for n samples."""
        if n < 2:
            return 1.0
        return np.sqrt(np.log(2 / self.delta) / (2 * n))

    def add_observation(self, error: float) -> None:
        """Add error observation for calibration."""
        self.samples.append(error)

    def decide(self) -> str:
        """Make fusion/fission decision with DKW guarantee."""
        n = len(self.samples)
        if n < self.min_samples:
            return self.current_state

        epsilon = self.dkw_epsilon(n)
        empirical_error = np.mean(self.samples[-self.min_samples:])
        error_upper_bound = empirical_error + epsilon

        if self.current_state == "fusion":
            if error_upper_bound > self.epsilon_target + self.hysteresis:
                self.current_state = "fission"
        else:
            if error_upper_bound < self.epsilon_target - self.hysteresis:
                self.current_state = "fusion"

        return self.current_state

# Test the controller with a simple example
controller = DKWController()
print(f"Initial state: {controller.current_state}")
print(f"DKW epsilon for 100 samples: {controller.dkw_epsilon(100):.4f}")
print(f"DKW epsilon for 1000 samples: {controller.dkw_epsilon(1000):.4f}")

In [ ]:
# Sample data mimicking GSM8K dataset structure
# In the real implementation, this would be: ds = load_dataset("gsm8k", "main", split="test[:200]")

sample_dataset = [
    {
        "question": "What is 2+2?",
        "answer": "2 + 2 = 4"
    },
    {
        "question": "If x=5, what is 2x?",
        "answer": "If x = 5, then 2x = 2 * 5 = 10"
    },
    {
        "question": "Solve: 3y + 6 = 15",
        "answer": "3y + 6 = 15\n3y = 15 - 6\n3y = 9\ny = 3"
    },
    {
        "question": "A store has 48 apples. If they sell 3/4 of them, how many apples are left?",
        "answer": "3/4 of 48 = (3/4) * 48 = 36\nApples left = 48 - 36 = 12"
    },
    {
        "question": "Calculate the area of a rectangle with length 8 cm and width 5 cm.",
        "answer": "Area = length * width = 8 * 5 = 40 square cm"
    }
]

print(f"Sample dataset loaded with {len(sample_dataset)} examples")

## 2. DKW Controller Class

The `DKWController` uses the **Dvoretzky-Kiefer-Wolfowitz inequality** to provide statistical guarantees on error rate estimates. Here's how it works:

### Key Parameters:
- `epsilon_target`: Target error rate threshold (10%)
- `delta`: Confidence parameter for DKW bound (5%)
- `min_samples`: Minimum samples before making decisions (100)
- `hysteresis`: Buffer to prevent mode oscillation (5%)

### Algorithm:
1. Collect error observations over time
2. Compute empirical error rate from recent samples
3. Use DKW inequality to get upper confidence bound on true error rate
4. Switch modes based on whether bound exceeds target (with hysteresis)

## Sample Data (Self-Contained)

Since this is a self-contained notebook, we'll use sample data instead of loading from HuggingFace. This mimics the structure of the GSM8K dataset.

In [ ]:
"""DKW Controller Implementation."""
import json
import numpy as np
from dataclasses import dataclass, field
import matplotlib.pyplot as plt
import pandas as pd

# Set random seed for reproducible results
np.random.seed(42)

In [ ]:
"""Dataset collection script for DKW benchmark."""
import json
# from datasets import load_dataset  # Would be used in full implementation

print("Libraries imported successfully!")

## 1. Import Dependencies

First, let's import the necessary libraries for our implementation:

## Imports and Setup

First, let's import the necessary libraries. In a full implementation, this would use the `datasets` library to load from HuggingFace, but here we'll work with sample data.

# DKW Controller Implementation Demo

This notebook demonstrates a **DKW-guided fusion/fission controller** - a statistical method for making decisions between fusion and fission modes based on error observations with confidence guarantees.

## Overview
- **DKW (Dvoretzky-Kiefer-Wolfowitz)** inequality provides statistical bounds on empirical distributions
- The controller switches between "fusion" and "fission" modes based on error rate observations
- Includes hysteresis to prevent oscillation between modes

# Dataset Collection Script for DKW Benchmark

This notebook demonstrates the dataset collection process for DKW controller evaluation. The original script collected data from the GSM8K dataset, but this version is self-contained with sample data inlined.

**Artifact Information:**
- ID: dataset_001
- Name: data.py
- Purpose: Collect and process benchmark data for evaluation

## How to Modify This Notebook

This notebook is completely self-contained and ready to use! Here's how you can customize it:

### 🔧 Modify the Data
- Edit the **Sample Data** cell to change the number of examples or error rates
- Adjust the fusion/fission decision ratios for either method
- Add more methods by extending the `results` dictionary

### 📊 Add New Metrics  
- Extend the `compute_metrics` function to calculate additional performance measures
- Add visualizations using matplotlib or seaborn
- Compare with other baseline methods

### 💾 Export Results
- The metrics are stored in the `eval_output` variable
- Use `json.dumps()` to save results to a file if needed
- Create CSV exports for spreadsheet analysis

**Original Script**: This notebook replaces the `eval.py` script and eliminates the need for external JSON files by inlining all data directly in the notebook.

In [ ]:
# Display detailed metrics in a readable format
print("=" * 60)
print("DETAILED EVALUATION RESULTS")
print("=" * 60)

print("\n📊 BASELINE METHOD:")
print(f"  Fusion Rate:     {metrics['baseline']['fusion_rate']:.1%}")
print(f"  Fission Rate:    {metrics['baseline']['fission_rate']:.1%}")  
print(f"  Error Rate:      {metrics['baseline']['error_rate']:.1%}")
print(f"  Total API Calls: {metrics['baseline']['api_calls']}")
print(f"  Avg Calls/Example: {metrics['baseline']['avg_calls_per_example']:.2f}")

print("\n🚀 PROPOSED METHOD:")
print(f"  Fusion Rate:     {metrics['proposed']['fusion_rate']:.1%}")
print(f"  Fission Rate:    {metrics['proposed']['fission_rate']:.1%}")
print(f"  Error Rate:      {metrics['proposed']['error_rate']:.1%}")  
print(f"  Total API Calls: {metrics['proposed']['api_calls']}")
print(f"  Avg Calls/Example: {metrics['proposed']['avg_calls_per_example']:.2f}")

print("\n📈 IMPROVEMENTS:")
print(f"  API Reduction:   {metrics['improvement']['api_reduction_pct']:.1f}%")
print(f"  Error Rate Change: {metrics['improvement']['error_rate_diff']:+.1%}")

print("\n" + "=" * 60)

# Also display the raw metrics as JSON for reference
print("\nRaw metrics (JSON format):")
print(json.dumps(metrics, indent=2))

## Detailed Results

Let's examine all the computed metrics in detail:

In [ ]:
# Compute metrics from our sample data
metrics = compute_metrics(results)

# Display key result (equivalent to the original script's print statement)
print(f"API reduction: {metrics['improvement']['api_reduction_pct']:.1f}%")

# Store the computed metrics for further analysis
# (This replaces the original file writing operation)
eval_output = metrics

## Run Evaluation

Now let's compute the metrics using our sample data and display the results. This replaces the original file I/O operations with direct computation.

In [ ]:
def compute_metrics(results: dict) -> dict:
    """Compute evaluation metrics for both baseline and proposed methods."""
    metrics = {}

    for method in ["baseline", "proposed"]:
        preds = results[method]

        # Count decisions
        fusion_count = sum(1 for p in preds if p["decision"] == "fusion")
        fission_count = sum(1 for p in preds if p["decision"] == "fission")

        # Compute error rate
        errors = sum(1 for p in preds if p["error"])
        error_rate = errors / len(preds)

        # API calls (fusion=1, fission=2)
        api_calls = fusion_count + 2 * fission_count

        metrics[method] = {
            "fusion_rate": fusion_count / len(preds),
            "fission_rate": fission_count / len(preds),
            "error_rate": error_rate,
            "api_calls": api_calls,
            "avg_calls_per_example": api_calls / len(preds),
        }

    # Compute improvement
    baseline_calls = metrics["baseline"]["avg_calls_per_example"]
    proposed_calls = metrics["proposed"]["avg_calls_per_example"]
    metrics["improvement"] = {
        "api_reduction_pct": (baseline_calls - proposed_calls) / baseline_calls * 100,
        "error_rate_diff": metrics["proposed"]["error_rate"] - metrics["baseline"]["error_rate"],
    }

    return metrics

print("Metrics computation function defined successfully!")

## Metrics Computation Function

The `compute_metrics` function analyzes the results from both methods and calculates:

1. **Decision Rates**: Percentage of fusion vs fission decisions
2. **Error Rates**: Percentage of examples that resulted in errors  
3. **API Efficiency**: Total API calls (fusion=1 call, fission=2 calls)
4. **Performance Comparison**: Improvement metrics between methods

In [ ]:
# Sample evaluation results data (inlined instead of reading from JSON files)
# This represents results from 200 test examples for both methods

# Create baseline results: all fission decisions, 8% error rate
baseline_results = []
for i in range(200):
    baseline_results.append({
        "decision": "fission",
        "error": i < 16  # First 16 examples have errors (8% error rate)
    })

# Create proposed method results: 65% fusion, 35% fission, 9% error rate  
proposed_results = []
for i in range(200):
    if i < 130:  # First 130 examples use fusion (65%)
        decision = "fusion"
    else:  # Remaining 70 examples use fission (35%)
        decision = "fission"
    
    proposed_results.append({
        "decision": decision,
        "error": i < 18  # First 18 examples have errors (9% error rate)
    })

# Combine into the expected data structure
results = {
    "baseline": baseline_results,
    "proposed": proposed_results
}

print(f"Created evaluation data:")
print(f"  Baseline examples: {len(results['baseline'])}")
print(f"  Proposed examples: {len(results['proposed'])}")
print(f"  Baseline fusion decisions: {sum(1 for p in results['baseline'] if p['decision'] == 'fusion')}")
print(f"  Proposed fusion decisions: {sum(1 for p in results['proposed'] if p['decision'] == 'fusion')}")
print(f"  Baseline errors: {sum(1 for p in results['baseline'] if p['error'])}")
print(f"  Proposed errors: {sum(1 for p in results['proposed'] if p['error'])}")

## Sample Data

Instead of reading from external JSON files, we'll define the evaluation data directly in the notebook. This represents the results of running both baseline and proposed methods on a test dataset.

In [ ]:
"""Evaluation script for DKW Controller."""
import json
import numpy as np
from typing import Dict, Any

print("Libraries imported successfully!")

## Overview

This evaluation script compares two methods:
- **Baseline**: Traditional approach that always uses fission decisions
- **Proposed**: Improved approach that intelligently chooses between fusion and fission

The metrics we compute include:
- Fusion/fission decision rates
- Error rates
- API call efficiency
- Performance improvements

# DKW Controller Evaluation

This notebook evaluates the performance of DKW Controller methods by comparing baseline and proposed approaches. It computes various metrics including fusion/fission rates, error rates, and API call efficiency.

## 🎯 Usage Instructions

This notebook is now completely self-contained and ready to use! Here's how to customize it:

### 🔧 Configuration Options:

1. **Data Source** (Cell 6):
   - Set `USE_LIVE_DATA = True` to fetch real data from HuggingFace GSM8K dataset
   - Set `USE_LIVE_DATA = False` to use the inlined sample data

2. **Export Format** (Cell 8):
   - `"display"` - Show JSON output in notebook (default)
   - `"json"` - Save to `data_out.json` file
   - `"csv"` - Save to `data_out.csv` file

### 🚀 Next Steps:

- Modify the difficulty calculation in the `collect_data()` function
- Add more data processing steps
- Integrate with your DKW benchmark evaluation pipeline
- Extend the sample data with more examples

### 📝 Notes:

- No external file dependencies required
- All data is either fetched from HuggingFace or inlined as Python objects
- Fully interactive with pandas DataFrame display
- Error handling included for network issues

**Enjoy your self-contained dataset collection notebook!** 🎉

In [ ]:
# Export options
def export_data(data, format_type="json"):
    """Export data in various formats."""
    if format_type == "json":
        # Original functionality - save as JSON
        with open("data_out.json", "w") as f:
            json.dump(data, f, indent=2)
        print(f"✅ Exported {len(data)} examples to data_out.json")
        
    elif format_type == "csv":
        # Export as CSV using pandas
        df.to_csv("data_out.csv", index=False)
        print(f"✅ Exported {len(data)} examples to data_out.csv")
        
    elif format_type == "display":
        # Just display the JSON in notebook
        print("📄 JSON Output:")
        print(json.dumps(data, indent=2))

# Choose export format
EXPORT_FORMAT = "display"  # Options: "json", "csv", "display"

if EXPORT_FORMAT in ["json", "csv"]:
    export_data(data, EXPORT_FORMAT)
else:
    export_data(data, "display")

print(f"\\n🎉 Data collection complete! Processed {len(data)} examples.")

## 💾 Data Export

Export the processed data to various formats. This replaces the original file writing functionality with more flexible export options.

In [ ]:
# Convert to pandas DataFrame for better display
df = pd.DataFrame(data)

# Display basic statistics
print("📈 Data Statistics:")
print(f"   Questions range from {df['difficulty'].min():.3f} to {df['difficulty'].max():.3f} difficulty")
print(f"   Question lengths: {df['question'].str.len().min()} - {df['question'].str.len().max()} characters")

# Display the data table
print(f"\n🗂️ Complete Dataset ({len(df)} rows):")
display(df)

# Show difficulty distribution
print(f"\n📊 Difficulty Distribution:")
difficulty_bins = pd.cut(df['difficulty'], bins=3, labels=['Easy', 'Medium', 'Hard'])
print(difficulty_bins.value_counts())

## 📋 Interactive Data Display

View the collected data in an interactive table format with sorting and filtering capabilities.

In [ ]:
# Choose your data source
USE_LIVE_DATA = False  # Set to True to fetch from HuggingFace, False to use sample data

if USE_LIVE_DATA:
    print("🌐 Collecting live data from HuggingFace GSM8K dataset...")
    try:
        data = collect_data()
        print(f"✅ Successfully collected {len(data)} examples from HuggingFace")
    except Exception as e:
        print(f"❌ Error loading live data: {e}")
        print("📋 Falling back to sample data...")
        data = sample_data
else:
    print("📋 Using inlined sample data...")
    data = sample_data

print(f"\nDataset Summary:")
print(f"📊 Total examples: {len(data)}")
print(f"💡 Average difficulty: {sum(item['difficulty'] for item in data) / len(data):.3f}")
print(f"📏 Average question length: {sum(len(item['question']) for item in data) / len(data):.1f} characters")

## 🚀 Run Data Collection

Execute the data collection function and display the results. You can choose to either:
1. Collect live data from HuggingFace (requires internet connection)
2. Use the inlined sample data for demonstration

In [ ]:
# Sample data inlined from data_out.json
sample_data = [
    {
        "id": "example_000",
        "question": "What is 2+2?",
        "answer": "4",
        "difficulty": 0.15
    },
    {
        "id": "example_001",
        "question": "If x=5, what is 2x?",
        "answer": "10",
        "difficulty": 0.22
    },
    {
        "id": "example_002",
        "question": "Solve: 3y + 6 = 15",
        "answer": "y=3",
        "difficulty": 0.28
    }
]

print(f"Sample dataset contains {len(sample_data)} examples")
print("Sample data structure:")
for item in sample_data[:1]:  # Show structure of first item
    for key, value in item.items():
        print(f"  {key}: {value} ({type(value).__name__})")

## 📊 Sample Data (Inlined)

Instead of reading from external JSON files, here's the sample data inlined for demonstration purposes. This makes the notebook completely self-contained.

In [ ]:
def collect_data():
    """Collect benchmark data for DKW controller evaluation."""
    # Load HuggingFace dataset
    print("Loading GSM8K dataset from HuggingFace...")
    ds = load_dataset("gsm8k", "main", split="test[:200]")
    print(f"Loaded {len(ds)} examples")

    data = []
    for i, example in enumerate(ds):
        data.append({
            "id": f"example_{i:03d}",
            "question": example["question"],
            "answer": example["answer"],
            "difficulty": len(example["question"]) / 100,  # Simple proxy based on question length
        })

    return data

## 🔧 Data Collection Function

The main function that processes the GSM8K dataset and creates structured benchmark data with difficulty scoring.

In [ ]:
"""Dataset collection script for DKW benchmark."""
import json
from datasets import load_dataset
import pandas as pd
from IPython.display import display, HTML

## 📦 Imports and Setup

# Dataset Collection Script for DKW Benchmark

**Artifact:** dataset_001 - data.py

This notebook contains a self-contained version of the dataset collection script for DKW benchmark evaluation. It loads and processes the GSM8K dataset to create benchmark data for controller evaluation.

## Features:
- Loads HuggingFace GSM8K dataset
- Processes examples with metadata (ID, difficulty score)
- Displays collected data in an interactive format
- Completely self-contained with inlined sample data

## Usage Notes

### Running this notebook:
1. **Self-contained mode**: Run all cells as-is to see sample data processing
2. **Live mode**: Uncomment the live data collection lines to fetch real GSM8K data

### Modifications you can make:
- Change the dataset split in `collect_data()` (e.g., `"test[:500]"` for more examples)
- Modify the difficulty calculation logic
- Add additional data processing steps
- Export results to different formats

### Original vs. Notebook differences:
- ✅ **Inlined data**: No dependency on `data_out.json` file
- ✅ **Interactive**: Can run sections independently
- ✅ **Documented**: Clear explanations for each step
- ✅ **Flexible**: Easy to modify and experiment with

**This notebook is completely self-contained and ready to run!**

In [ ]:
# Display the collected data in a readable format
for i, example in enumerate(data):
    print(f"\n--- Example {i+1} ---")
    print(f"ID: {example['id']}")
    print(f"Question: {example['question']}")
    print(f"Answer: {example['answer']}")
    print(f"Difficulty: {example['difficulty']:.2f}")

print(f"\n✅ Successfully processed {len(data)} examples")
print("📝 Data structure matches the original script output")
print("🔄 Notebook is completely self-contained - no external files needed!")

In [ ]:
# Option 1: Live data collection (uncomment to run with internet access)
# print("Collecting live data from HuggingFace...")
# live_data = collect_data()
# print(f"Collected {len(live_data)} examples from GSM8K dataset")

# Option 2: Use sample data for demonstration (self-contained)
print("Using sample data for demonstration:")
data = sample_output_data

# Original script would save to file - here we'll just display
# with open("data_out.json", "w") as f:
#     json.dump(data, f, indent=2)

print(f"Collected {len(data)} examples")
print("\nFirst few examples:")

## Execute Data Collection

Now let's run the data collection function. In the original script, this would load from HuggingFace and save to a file. Here we'll demonstrate both approaches:

1. **Live data collection** (requires internet and HuggingFace datasets)
2. **Display of sample data** (self-contained)

In [ ]:
# Inlined sample data (replaces data_out.json for self-contained execution)
sample_output_data = [
    {
        "id": "example_000",
        "question": "What is 2+2?",
        "answer": "4",
        "difficulty": 0.15
    },
    {
        "id": "example_001",
        "question": "If x=5, what is 2x?",
        "answer": "10",
        "difficulty": 0.22
    },
    {
        "id": "example_002",
        "question": "Solve: 3y + 6 = 15",
        "answer": "y=3",
        "difficulty": 0.28
    }
]

print(f"Sample data contains {len(sample_output_data)} examples")

## Self-Contained Data (No External Dependencies)

Instead of reading from `data_out.json`, we'll inline the sample data here to make this notebook completely self-contained. This demonstrates what the output would look like without requiring external files.

In [ ]:
def collect_data():
    """Collect benchmark data for DKW controller evaluation."""
    # Load HuggingFace dataset
    ds = load_dataset("gsm8k", "main", split="test[:200]")

    data = []
    for i, example in enumerate(ds):
        data.append({
            "id": f"example_{i:03d}",
            "question": example["question"],
            "answer": example["answer"],
            "difficulty": len(example["question"]) / 100,  # Simple proxy
        })

    return data

In [ ]:
"""Dataset collection script for DKW benchmark."""
import json
from datasets import load_dataset

## Overview

This notebook demonstrates:
1. **Data Collection**: Loading benchmark data from HuggingFace's GSM8K dataset
2. **Data Processing**: Formatting examples with IDs, questions, answers, and difficulty scores
3. **Self-contained execution**: All data is inlined to eliminate external dependencies

The original script would save data to `data_out.json` - here we'll display the results directly and provide the data inline for demonstration.

# Dataset Collection for DKW Benchmark

**Artifact:** dataset_001 (data.py)

This notebook contains a self-contained version of the dataset collection script for DKW benchmark evaluation. The original script has been converted to run entirely within this notebook without any external file dependencies.